In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [32]:
dataset=pd.read_csv('/content/Loan_default.csv.zip')

In [20]:
dataset.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [33]:
dataset.drop(['LoanID', 'NumCreditLines', 'DTIRatio', 'MaritalStatus', 'HasMortgage', 'LoanPurpose', 'HasCoSigner'], axis = 1, inplace = True)

In [23]:
dataset

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,InterestRate,LoanTerm,Education,EmploymentType,HasDependents,Default
0,56,85994,50587,520,80,15.23,36,Bachelor's,Full-time,Yes,0
1,69,50432,124440,458,15,4.81,60,Master's,Full-time,No,0
2,46,84208,129188,451,26,21.17,24,Master's,Unemployed,Yes,1
3,32,31713,44799,743,0,7.07,24,High School,Full-time,No,0
4,60,20437,9139,633,8,6.51,48,Bachelor's,Unemployed,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...
255342,19,37979,210682,541,109,14.11,12,Bachelor's,Full-time,No,0
255343,32,51953,189899,511,14,11.55,24,High School,Part-time,No,1
255344,56,84820,208294,597,70,5.29,60,High School,Self-employed,Yes,0
255345,42,85109,60575,809,40,20.90,48,High School,Part-time,Yes,0


In [25]:
dataset.value_counts('Default')

,count
Default,
0,225694
1,29653


In [27]:
dataset.value_counts('Education')

,count
Education,
Bachelor's,64366
High School,63903
Master's,63541
PhD,63537


In [28]:
dataset.value_counts('EmploymentType')

,count
EmploymentType,
Part-time,64161
Unemployed,63824
Self-employed,63706
Full-time,63656


In [29]:
dataset.value_counts('HasDependents')

,count
HasDependents,
Yes,127742
No,127605


In [34]:
dataset['HasDependents']=dataset['HasDependents'].replace({'Yes':1,'No':0})

<ipython-input-34-2291bead5222>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['HasDependents']=dataset['HasDependents'].replace({'Yes':1,'No':0})


In [35]:
dataset

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,InterestRate,LoanTerm,Education,EmploymentType,HasDependents,Default
0,56,85994,50587,520,80,15.23,36,Bachelor's,Full-time,1,0
1,69,50432,124440,458,15,4.81,60,Master's,Full-time,0,0
2,46,84208,129188,451,26,21.17,24,Master's,Unemployed,1,1
3,32,31713,44799,743,0,7.07,24,High School,Full-time,0,0
4,60,20437,9139,633,8,6.51,48,Bachelor's,Unemployed,1,0
...,...,...,...,...,...,...,...,...,...,...,...
255342,19,37979,210682,541,109,14.11,12,Bachelor's,Full-time,0,0
255343,32,51953,189899,511,14,11.55,24,High School,Part-time,0,1
255344,56,84820,208294,597,70,5.29,60,High School,Self-employed,1,0
255345,42,85109,60575,809,40,20.90,48,High School,Part-time,1,0


In [36]:
dataset['EmploymentType'] = dataset['EmploymentType'].replace({'Unemployed' : 0, 'Part-time' : 1, 'Full-time' : 2, 'Self-employed' : 3})

<ipython-input-36-6337caae5093>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['EmploymentType']=dataset['EmploymentType'].replace({'Unemployed' : 0, 'Part-time' : 1, 'Full-time' : 2, 'Self-employed' : 3})


In [37]:
dataset

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,InterestRate,LoanTerm,Education,EmploymentType,HasDependents,Default
0,56,85994,50587,520,80,15.23,36,Bachelor's,2,1,0
1,69,50432,124440,458,15,4.81,60,Master's,2,0,0
2,46,84208,129188,451,26,21.17,24,Master's,0,1,1
3,32,31713,44799,743,0,7.07,24,High School,2,0,0
4,60,20437,9139,633,8,6.51,48,Bachelor's,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
255342,19,37979,210682,541,109,14.11,12,Bachelor's,2,0,0
255343,32,51953,189899,511,14,11.55,24,High School,1,0,1
255344,56,84820,208294,597,70,5.29,60,High School,3,1,0
255345,42,85109,60575,809,40,20.90,48,High School,1,1,0


In [38]:
dataset['Education'] = dataset['Education'].replace({'High School' : 0, "Bachelor's" : 1, "Master's" : 2, 'PhD' : 3})

<ipython-input-38-95c7810eb4f1>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['Education'] = dataset['Education'].replace({'High School' : 0, "Bachelor's" : 1, "Master's" : 2, 'PhD' : 3})


In [39]:
dataset

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,InterestRate,LoanTerm,Education,EmploymentType,HasDependents,Default
0,56,85994,50587,520,80,15.23,36,1,2,1,0
1,69,50432,124440,458,15,4.81,60,2,2,0,0
2,46,84208,129188,451,26,21.17,24,2,0,1,1
3,32,31713,44799,743,0,7.07,24,0,2,0,0
4,60,20437,9139,633,8,6.51,48,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
255342,19,37979,210682,541,109,14.11,12,1,2,0,0
255343,32,51953,189899,511,14,11.55,24,0,1,0,1
255344,56,84820,208294,597,70,5.29,60,0,3,1,0
255345,42,85109,60575,809,40,20.90,48,0,1,1,0


In [41]:
X=dataset.drop(['Default'],axis=1)
Y=dataset['Default']

In [42]:
X.shape

(255347, 10)

In [43]:
X.median()

,0
Age,43.00
Income,82466.00
LoanAmount,127556.00
CreditScore,574.00
MonthsEmployed,60.00
InterestRate,13.46
LoanTerm,36.00
Education,1.00
EmploymentType,1.00
HasDependents,1.00


In [44]:
scale=StandardScaler()

In [45]:
scaled_X=scale.fit_transform(X)

In [49]:
X_train,X_test,Y_train,Y_test=train_test_split(scaled_X,Y,test_size=0.2,random_state=31)

In [50]:
model=LogisticRegression()

In [51]:
model.fit(X_train,Y_train)

LogisticRegression()

In [52]:
prediction_on_training_data=model.predict(X_train)
accuracy_on_training_data=accuracy_score(prediction_on_training_data,Y_train)
print("the accuracy score on training data: ",accuracy_on_training_data)

the accuracy score on training data:  0.8847692104348507


In [53]:
prediction_on_testing_data=model.predict(X_test)
accuracy_on_testing_data=accuracy_score(prediction_on_testing_data,Y_test)
print("the accuracy score on testing data: ",accuracy_on_testing_data)

the accuracy score on testing data:  0.884354807127472


In [55]:
model2=RandomForestClassifier()

In [56]:
model2.fit(X_train,Y_train)

RandomForestClassifier()

In [57]:
prediction_on_training_data=model2.predict(X_train)
accuracy_on_training_data=accuracy_score(prediction_on_training_data,Y_train)
print("the accuracy score on training data: ",accuracy_on_training_data)

the accuracy score on training data:  0.9999559421765544


In [58]:
prediction_on_testing_data=model2.predict(X_test)
accuracy_on_testing_data=accuracy_score(prediction_on_testing_data,Y_test)
print("the accuracy score on testing data: ",accuracy_on_testing_data)

the accuracy score on testing data:  0.8851967887213629


In [60]:
model3=XGBClassifier()

In [61]:
model3.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [62]:
prediction_on_training_data=model3.predict(X_train)
accuracy_on_training_data=accuracy_score(prediction_on_training_data,Y_train)
print("the accuracy score on training data: ",accuracy_on_training_data)

the accuracy score on training data:  0.8924107951458069


In [63]:
prediction_on_testing_data=model3.predict(X_test)
accuracy_on_testing_data=accuracy_score(prediction_on_testing_data,Y_test)
print("the accuracy score on testing data: ",accuracy_on_testing_data)

the accuracy score on testing data:  0.8846485216369688


In [64]:
import pickle

In [65]:
pickle.dump(model3,open('/content/Loan_Predictor.pkl','wb'))